# Práctica 1 - Self-Organising Maps - COLORES
## Preparación de entorno
#### Importar librerías de código

In [29]:
# from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
from sklearn import preprocessing

%matplotlib inline

#### Dataset que se va a utilizar para el entrenamiento

In [30]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
valor_max = 256
valor_min = 0
valores_color = 3
num_colores = 100
datos = np.random.randint(valor_min, valor_max, (valores_color, num_colores))

## SOM Setup
#### Variables definidas por el alumno

In [31]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 100
periodo = 1000
learning_rate = 0.2
normalizar_datos = True

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [32]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
num_entradas = valores_color
num_datos = num_colores

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario_inicial = int(lado_mapa/2)

# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
if normalizar_datos:
    datos = datos / 255
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
matriz_pesos = np.random.random((lado_mapa,lado_mapa,valores_color))

#### Funciones para entrenar/clasificar

In [33]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):
   rows = len(m_pesos)
   columns = len(m_pesos[0])
   vectorPesos =  []
   ganadora = 2000
   for x in range(rows):
      for y in range(columns):
         for z in range(m):
            vectorPesos.append(m_pesos[x][y][z])
         dist = np.linalg.norm(np.array(vectorPesos) - np.array(patron_entrada))
         if dist < ganadora:
            ganadora = dist
            Bmu = vectorPesos
            BmuIndex = np.stack((x,y))
         vectorPesos = []
   return Bmu , BmuIndex

In [34]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
   learning_rate = lr_inicial * (1 - i/n_iteraciones)
   return learning_rate


In [35]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
   vecindario = 1 + vecindario_inicial * (1 - i/n_iteraciones)
   return vecindario
   

In [36]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

In [37]:
#Funcion Ajuste de matriz de pesos
def ajustarPesos(IndexN , color , learning_rate , matriz_pesos , vecindario):
    matriz_pesos[IndexN[0]][IndexN[1]] += (learning_rate * (color - matriz_pesos[IndexN[0]][IndexN[1]]))
    rows = len(matriz_pesos)
    cols = len(matriz_pesos[0])
    for x in range(rows):
        for y in range(cols):
            vectorNeurona = [x,y]
            vectorBMU = [IndexN[0],IndexN[1]]
            dist_euc = np.linalg.norm(np.array(vectorBMU) - np.array(vectorNeurona))
            if(dist_euc <= vecindario and dist_euc != 0):
              dist_decay = decay(dist_euc,vecindario)
              matriz_pesos[x][y] += (learning_rate * dist_decay * (color -  matriz_pesos[x][y]))  
            
    return matriz_pesos


#### Funciones para dibujar la salida de la red

In [38]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % periodo)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

In [39]:
def shuffle():
    rng = np.random.default_rng()
    datos = datos = np.random.randint(valor_min, valor_max, (valores_color, num_colores))
    datos = datos/255
    datos = np.transpose(datos)
    datos = rng.permuted(datos, out=datos) # MOvemos los datos pa que no se acostumbre
    return datos

## SOM Entrenamiento

In [40]:
# Entrena la red con el dataset de entrenamiento
pintar_mapa(matriz_pesos)
datosT = np.transpose(datos)
vecindario = vecindario_inicial
learning_rate_ajustado = learning_rate
for x in range(periodo):
    if(x % 100 == 0):
        print(x, "iteraciones")
    numAleatorio = np.random.randint(0, num_colores)
    color = datosT[numAleatorio]
    Bmu , BmuIndex = calcular_bmu(color,matriz_pesos,valores_color)
    matriz_pesos = ajustarPesos(BmuIndex , color , learning_rate_ajustado , matriz_pesos , vecindario)
    learning_rate_ajustado = variacion_learning_rate(learning_rate, x , periodo)
    vecindario = variacion_vecindario(vecindario_inicial , x , periodo)
pintar_mapa(matriz_pesos)

KeyboardInterrupt: 

## SOM Clasificación

In [ ]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada



## SOM Prueba

In [ ]:
# Clasifica nuevos patrones

